<a href="https://colab.research.google.com/github/vitfrncs/LiPAI_board/blob/master/src/semana_9/Semana9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semana 9 - Data augmentation

-----
1 - Modifique a etapa de transformações (trans) para incluir uma rotação aleatória nas imagens de treinamento. Utilize torchvision.transforms.RandomRotation com um ângulo de até 15 graus. Como ficaria a nova definição de transforms.Compose?

A etapa de transformações foi modificada para incluir uma rotação aleatória de até 15° nas imagens do conjunto de treinamento, utilizando torchvision.transforms.RandomRotation(15), enquanto o conjunto de teste permaneceu apenas com ToTensor e Normalize, garantindo que a rotação seja aplicada apenas durante o treinamento como técnica de data augmentation.

```python
import torch

import torchvision
from torchvision import transforms

# MNIST dataset
root_path = '/home/storopoli/Downloads' # mude isso no Colab se necessário

#QUESTAO 1:
# Pequena transformação para tensores e normalizando o tamanho
#trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# transformações para o conjunto de treinamento
train_trans = transforms.Compose([
    transforms.RandomRotation(15),                      # rotação aleatória até ±15°
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# transformações para o conjunto de teste (sem rotação!)
test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])


# Train/Test Datasets
# como estava antes:
# train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans, download=True)
# test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=trans)

# como ficou:
train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=train_trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=test_trans)


---
2 - Construa uma nova estratégia de transformação chamada trans_aumentado que aplique sequencialmente as seguintes técnicas de aumento de dados:
- Translação horizontal e vertical aleatória de até 10% da dimensão da imagem (RandomAffine).
- Zoom aleatório, variando a escala da imagem entre 90% e 110% (RandomAffine).
- Alteração de perspectiva (RandomPerspective).

Após criar essa estratégia, substitua a etapa de transformação original no train_dataset e treine o modelo novamente.

In [3]:
import torch

import torchvision
from torchvision import transforms

# MNIST dataset
root_path = '/home/storopoli/Downloads' # mude isso no Colab se necessário

# Pequena transformação para tensores e normalizando o tamanho
#trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

#QUESTAO 2:
trans_aumentado = transforms.Compose([
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),   # translação até 10%
    transforms.RandomAffine(degrees=0, scale=(0.9, 1.1)),       # zoom 90% a 110%
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # alteração de perspectiva
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Train/Test Datasets
# como estava antes:
# train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans, download=True)
# test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=trans)

# como ficou:
train_dataset = torchvision.datasets.MNIST(root=root_path, train=True, transform=trans_aumentado, download=True)
test_dataset = torchvision.datasets.MNIST(root=root_path, train=False, transform=test_trans)


----
3 - A transformação transforms.RandomHorizontalFlip() é muito comum em problemas de visão computacional. Você a aplicaria para o dataset MNIST? Justifique sua resposta pensando em como essa transformação afetaria a classificação de dígitos específicos (por exemplo, o '6' e o '9').

Não seria adequado aplicar transforms.RandomHorizontalFlip() no dataset MNIST, pois a inversão horizontal pode alterar completamente o significado de alguns dígitos, como transformar um “6” em algo semelhante a um “9” ou distorcer o “2” e o “5”, o que introduziria ruído nos dados em vez de melhorar a capacidade de generalização do modelo.

In [4]:
from torch.utils.data import DataLoader

batch_size=32

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
import torch.nn as nn

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Sequential(
            nn.Linear(7 * 7 * 64, 1000),
            nn.ReLU())
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

# Instancia o Model()
model = ConvNet()

print(model)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=3136, out_features=1000, bias=True)
    (1): ReLU()
  )
  (fc2): Linear(in_features=1000, out_features=10, bias=True)
)


In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

3199106

In [8]:
from torch.optim import Adam

# Hiperparâmetros
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 6

# Instânciar o Otimizador Adam
optimizer = Adam(model.parameters(), lr=learning_rate)

In [9]:
# Isto tem que retornar True
import torch
torch.cuda.is_available()

True

In [10]:
# Sua GPU
torch.cuda.get_device_name()

'Tesla T4'

In [11]:
# Treinar o Modelo
total_step = len(train_loader) # quantos batches eu tenho

# Listas vazias
loss_list = []
acc_list = []

for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Gera a propagação (feed forward)
        outputs = model(images)

        # Calcula a função-custo
        loss = loss_fn(outputs, labels)
        loss_list.append(loss.item())

        # Retro-propagação (Backprop) e a otimização com Adam
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Acurácia
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        if (i + 1) % 100 == 0:
            print(f"Época [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], Custo: {round(loss.item(), 3)}, Acurácia: {round((correct / total) * 100, 3)}")

Época [1/6], Step [100/1875], Custo: 0.423, Acurácia: 87.5
Época [1/6], Step [200/1875], Custo: 0.404, Acurácia: 84.375
Época [1/6], Step [300/1875], Custo: 0.309, Acurácia: 84.375
Época [1/6], Step [400/1875], Custo: 0.146, Acurácia: 96.875
Época [1/6], Step [500/1875], Custo: 0.154, Acurácia: 93.75
Época [1/6], Step [600/1875], Custo: 0.24, Acurácia: 93.75
Época [1/6], Step [700/1875], Custo: 0.27, Acurácia: 90.625
Época [1/6], Step [800/1875], Custo: 0.108, Acurácia: 96.875
Época [1/6], Step [900/1875], Custo: 0.037, Acurácia: 100.0
Época [1/6], Step [1000/1875], Custo: 0.07, Acurácia: 96.875
Época [1/6], Step [1100/1875], Custo: 0.2, Acurácia: 93.75
Época [1/6], Step [1200/1875], Custo: 0.066, Acurácia: 96.875
Época [1/6], Step [1300/1875], Custo: 0.171, Acurácia: 90.625
Época [1/6], Step [1400/1875], Custo: 0.096, Acurácia: 96.875
Época [1/6], Step [1500/1875], Custo: 0.508, Acurácia: 93.75
Época [1/6], Step [1600/1875], Custo: 0.076, Acurácia: 96.875
Época [1/6], Step [1700/1875]

In [12]:
model.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        # Feed-forward com as imagens de teste
        outputs = model(images)

        # gera predições usando a função max()
        _, predicted = torch.max(outputs.data, 1)

        # Acumula total e corretas
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Acurácia do Modelo em 10k imagens de teste: {round((correct / total) * 100, 3)}")

Acurácia do Modelo em 10k imagens de teste: 99.19
